## Step 1. 데이터 다운로드
- <a href="https://github.com/songys/Chatbot_data" target="_blank"> songys/Chatbot_data </a>

#### 심볼릭 링크 디렉토리 생성

- mkdir -p ~/aiffel/transformer_chatbot/data
- ln -s ~/data/* ~/aiffel/transformer_chatbot/data

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split
from konlpy.tag import Mecab
from tqdm import tqdm
print(tf.__version__)

2.4.1


In [2]:
# [E-15] CHAT-BOT 노드 참고 
dataset_filepath = os.getenv('HOME') + '/aiffel/transformer_chatbot/data/ChatbotData .csv'
df = pd.read_csv(dataset_filepath)

df.head(5)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
print('전체 샘플수 :', (len(df)))

전체 샘플수 : 11823


#### 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장

In [4]:
# 데이터를 로드하고 전처리하여 질문을 questions, 답변을 answers에 저장합니다.
questions = df['Q']
answers = df['A']

In [5]:
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))

전체 샘플 수 : 11823
전체 샘플 수 : 11823


## Step 2. 데이터 정제
- 영문자의 경우, 모두 소문자로 변환합니다.
- 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.

In [6]:
# 노드 8. Seq2seq으로 번역기 만들기, 12노드 참고- 데이터 정제하기

def preprocess_sentence(sentence):  #preprocess_sentence :  문장부호와 대소문자등을 정제하는 함수
    
    sentence = sentence.lower()   # 모든 입력을 소문자로 변환
   
    sentence = re.sub(r"[^a-zA-Z가-힣?.!,]+", " ", sentence)  # 알파벳, 한글, 문장부호만 남기고 모두 제거
    
    sentence = '<start> ' + sentence + ' <end>' # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

In [7]:
# 전처리 진행 확인

print('전처리 후의 22번째 질문 샘플: {}'.format(questions[13]))
print('전처리 후의 22번째 답변 샘플: {}'.format(answers[13]))

전처리 후의 22번째 질문 샘플: 가끔은 혼자인게 좋다
전처리 후의 22번째 답변 샘플: 혼자를 즐기세요.


## Step 3. 데이터 토큰화
- 토큰화에는 KoNLPy의 mecab 클래스를 사용
- build_corpus() 함수를 구현

In [ ]:
소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.   질문만 하는 나라의 언어를 Source언어 / 답변만 하는 나라의 언어를 Target 언어
데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다.
토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외합니다.
중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!
구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [ ]:
# 위에서 만든 전처리 함수에서 소스 문장을 제거하면 타겟 문장이 된다. 

corpus = []

# 모든 문장에 전처리 함수 적용
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    corpus.append(preprocess_sentence(sentence))

corpus[:10]

## Step 4. Augmentation  11노드 어휘대체

In [ ]:
우리에게 주어진 데이터는 1만 개가량으로 적은 편에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해 봐야겠죠? Lexical Substitution을 실제로 적용해 보도록 하겠습니다.

아래 링크를 참고하여 한국어로 사전 훈련된 Embedding 모델을 다운로드합니다. Korean (w) 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 Korean (w)를 찾아 다운로드하고, ko.bin 파일을 얻으세요!

Kyubyong/wordvectors
다운로드한 모델을 활용해 데이터를 Augmentation 하세요! 앞서 정의한 lexical_sub() 함수를 참고하면 도움이 많이 될 겁니다.

Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.

## Step 5. 데이터 벡터화

In [ ]:
타겟 데이터인 ans_corpus 에 <start> 토큰과 <end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다. 우리가 구축한 ans_corpus 는 list 형태이기 때문에 아주 쉽게 이를 해결할 수 있답니다!

In [ ]:
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])

## Step 6. 훈련하기

In [ ]:
앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! 가장 멋진 답변과 모델의 하이퍼파라미터를 제출하시면 됩니다.

## Step 7. 성능 측정하기

In [ ]:
챗봇의 경우, 올바른 대답을 하는지가 중요한 평가 지표입니다. 올바른 답변을 하는지 눈으로 확인할 수 있겠지만, 많은 데이터의 경우는 모든 결과를 확인할 수 없을 것입니다. 주어진 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산하는 calculate_bleu() 함수도 적용해 보세요.